# Encoding sentences using Google BERT

### Setup

In [1]:
import sys
sys.path.append('../')
from src.embeddings import BertEncoding
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tqdm import tqdm
import torch

# Libraries options
tqdm.pandas()
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 1000)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

/home/carina/anaconda3/envs/mbert_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


### Data Acess

In [2]:
# lendo o dado processado no notebook Datasets/data_treatment.ipynb
dataframe = pd.read_parquet('../Datasets/pre_processed_dataframes/clean_df.parquet')
len(dataframe)

5070

In [3]:
dataframe[['description', 'cleaned_description']].sample()

,description,cleaned_description
5021,"EXCLUSIVE: As the Santa Fe Sheriff’s Office continues their investigation of the fatal shooting of cinematographer Halyna Hutchins on October 21, which also injured director Joel Souza, on the set of the Alec Baldwin movie Rust, new claims and details are emerging of first assistant director, David Halls – including a 2019 complaint against him. […]",exclusive santa fe sheriffs office continues investigation fatal shooting cinematographer halyna hutchins october also injured director joel souza set alec baldwin movie rust new claims details emerging first assistant director david halls including complaint


In [4]:
dataframe[['full_description', 'cleaned_full_description']].sample()


,full_description,cleaned_full_description
1483,"(Teleborsa) – “La nostra intesa per lo smart working , in partnership con le associazioni sindacali, rappresenta un accordo importante e innovativo, che introduce un nuovo modo di lavorare e un modello culturale basato su fiducia e responsabilità”. Lo ha dichiarato Rossella Gangi , Direttrice Human Resources di WINDTRE , azienda guidata da Jeffrey Hedberg. “L’accordo, ha ricordato la manager, ridefinisce il lavoro da remoto in ottica post-emergenziale e lo caratterizza con un elevato livello di flessibilità . L’impostazione adottata da WINDTRE, infatti, lascia al singolo dipendente la libertà di scelta sulla modalità di lavoro, smart o in ufficio, in base alle esigenze personali e professionali. Tra i punti di forza del nostro modello – ricorda la Direttrice HR – la creazione della ‘We-etiquette’ , una carta dei buoni comportamenti per collaborare al meglio anche a distanza, e la grande attenzione riservata alla formazione attraverso programmi di Learning & Development. Iniziative ...",teleborsa intesa smart working partnership associazioni sindacali rappresenta accordo importante innovativo introduce nuovo modo lavorare modello culturale basato fiducia responsabilit dichiarato rossella gangi direttrice human resources windtre azienda guidata jeffrey hedberg laccordo ricordato manager ridefinisce lavoro remoto ottica postemergenziale caratterizza elevato livello flessibilit limpostazione adottata windtre infatti lascia singolo dipendente libert scelta modalit lavoro smart ufficio base esigenze personali professionali punti forza modello ricorda direttrice hr creazione weetiquette carta buoni comportamenti collaborare meglio distanza grande attenzione riservata formazione attraverso programmi learning development iniziative mirate fornire persone windtre strumenti adeguati affrontare cambiamento mondo lavoro accrescere competenze digitali modello smart working conclude rossella gangi oltre essere accolto grande soddisfazione dipendenti ricevuto importanti riconosc...


### Encoding Data

In [5]:
bert = BertEncoding(device=device)

In [ ]:
embeddings_1 = bert.encode('trabalhando com codificação e similaridade de texto')
embeddings_2 = bert.encode('working on coding and text similarity')

cosine_similarity([embeddings_1], [embeddings_2])[0][0]

In [7]:
dataframe['embeddings_description'] = dataframe['cleaned_description'].progress_apply(bert.encode)

100%|██████████| 5070/5070 [03:32<00:00, 23.89it/s]


In [8]:
dataframe['embeddings_full_description'] = dataframe['cleaned_full_description'].progress_apply(bert.encode)

100%|██████████| 5070/5070 [23:53<00:00,  3.54it/s] 


In [11]:
dataframe[['embeddings_description', 'cleaned_description']].sample()

,embeddings_description,cleaned_description
3335,"[0.12129478, 0.017495077, 0.13084859, 0.1385743, 0.20463625, 0.35059196, 0.14564255, -0.02975083, -0.09653957, 0.20360485, -0.16861236, -0.17020656, 0.22392271, -0.13363275, -0.18964843, 0.07130557, 0.10810741, 0.0735891, 0.18609786, 0.12309707, -0.062111773, -0.071568795, 0.053872, 0.043881014, 0.2449765, -0.11949419, 0.21694256, 0.118634656, 0.20315917, 0.26103145, 0.20579271, 0.21130861, 0.21956089, -0.086608715, 0.17354763, -0.03406146, 0.021359544, 0.11826823, 0.21701092, 0.045650426, 0.09293463, 0.3071541, 0.12200438, -0.079055, -0.26207194, 0.14071688, -0.16075563, -0.030006187, 0.99998134, 0.15233167, 0.07091681, -0.035701122, 0.07090309, -0.24892995, 0.23342583, 0.9999866, -0.3036685, -0.20367947, 0.042285264, -0.088647835, -0.18418156, 0.060812943, 0.26140308, 0.12317315, -0.07645892, 0.071928024, -0.071786776, 0.3017361, 0.06598939, 0.10719946, 0.10001891, -0.0671344, 0.19548897, 0.25544488, -0.099987336, 0.038017686, -0.13127266, -0.123172574, -0.1548261, 0.1698511, 0.1...",sils radiqus rapidement durablement termites peuvent provoquer gros dgts


#### Save Processed Dataframe

In [12]:
dataframe.to_parquet('../Datasets/pre_processed_dataframes/embeddings_df.parquet', index=False)